In [2]:
import pandas as pd
import numpy as np

import networkx as nx

# This uses the Answers.csv file from the 10% Stack Overflow data
answer_file = "data/Answers.csv"
# This edge list is the intermediate file used for graph building
edges_list_file = "processed_data/answer_edges.txt"

## Pre-processing

In [3]:
# loads data with pands, it eats up memory, but parsing with pyspark is much more work
df = pd.read_csv("data/Answers.csv", encoding="ISO-8859-1")
df.head(5)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [4]:
df.shape

(2014516, 6)

In [5]:
df.loc[df['ParentId'] == 90]


,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
10748,202317,20709.0,2008-10-14T18:41:45Z,90,2,"<p>You can also try <em><a href=""http://www.co..."
85572,1466832,16012.0,2009-09-23T15:40:46Z,90,19,<p>My easy click-by-click instructions (<stron...


In [6]:
# Question_ids and user_ids may overlap, but that does not mean questions are users!!!
# Soln: each question_id += max_user_id
max_user_id = df[['OwnerUserId']].max()
max_user_id

OwnerUserId    7045028.0
dtype: float64

In [7]:
edge_df = df[['OwnerUserId', 'ParentId']]
# 1. drop null values
edge_df = edge_df.dropna()
# 2. make parentIds unique
edge_df = edge_df.assign(newParentId=lambda x: x.ParentId + max(max_user_id))
edge_df = edge_df.drop(['ParentId'], axis=1)
# 3. add weights to edges
edge_df['EdgeWeight'] = 1
# 4. cast the datafraem to int type
edge_df = edge_df.astype('int32')
edge_df.head(30)

,OwnerUserId,newParentId,EdgeWeight
0,61,7045118,1
1,26,7045108,1
2,50,7045208,1
3,91,7045288,1
4,49,7045288,1
5,59,7045358,1
6,100,7045288,1
7,119,7045288,1
8,49,7045498,1
9,86,7045208,1


In [8]:
edge_df.loc[edge_df['newParentId'] == 7045118]

,OwnerUserId,newParentId,EdgeWeight
0,61,7045118,1
10748,20709,7045118,1
85572,16012,7045118,1


In [9]:
edge_df.to_csv('processed_data/answer_edges.txt', sep=' ', header=False, index=False)


## Build Graph

In [10]:
# by default, nx creates undirected edges, exactly what we want
G = nx.read_edgelist(edges_list_file, nodetype=int, data=(('weight',float),))
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1568735
Number of edges: 1993272
Average degree:   2.5412


In [11]:
all_user_ids = set()
all_question_ids = set()
with open(edges_list_file, 'r') as read_file:
    for line in read_file.readlines():
        user_id, question_id, weight = line.strip().split(' ')
        all_user_ids.add(int(user_id))
        all_question_ids.add(int(question_id))
print(list(all_user_ids)[:10])
print(list(all_question_ids)[:10])
# should be no intersection between user_ids and question_ids
print(len(all_user_ids.intersection(all_question_ids)))


[1, 3, 4, 5, 1048579, 2097159, 5242883, 9, 3145739, 1048588]
[18874368, 39845888, 37748738, 25165828, 35651588, 12582918, 46137348, 41943048, 8388618, 14680078]
0


In [12]:
# General Data Analysis
islands = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
print("connected components", islands[:10])

connected components [1328544, 53, 36, 34, 34, 29, 28, 27, 25, 24]


In [13]:
# analyze how connected the graph is
# connectivity of 0..... oh well
from networkx.algorithms import approximation as approx
approx.node_connectivity(G)

0

## Test Performance

In [25]:
import random
from collections import Counter
# parameters
n_test_edge = 1000
n_steps = 1000
teleportation_alpha = 0.5
early_stop_threshold = 20

# returns whether x2 - y2 edge is the same as x1 - y1 edge
# is_same_edge(1, 2, 2, 1) == is_same_edge(1, 2, 1, 2) == True
def is_same_edge(x1, y1, x2, y2):
    if x1 == x2:
        return y1 == y2
    elif x1 == y2:
        return y1 == x2
    return False

# a random walk on the user_node (x or y)
# pretend the direct edge (x, y) does not exist
# returns a distribution of questions nodes
def random_walk_on_edge(x, y):
    # curr_pos is always on user nodes
    starting_pos = x if x in all_user_ids else y
    curr_pos = starting_pos
    reacheable_count = Counter()
    for s in range(n_steps):
        try:
            potential_questions_nodes = random.sample(set(G[curr_pos]), 2)
            question_node = potential_questions_nodes[0] if not is_same_edge(x, y, potential_questions_nodes[0], curr_pos) else potential_questions_nodes[1]
            # diff from pinterest algorithm in that we care about questions, not users
            reacheable_count[question_node] += 1
            if sum(reacheable_count.values()) / len(reacheable_count.values()) >= early_stop_threshold:
                # if average > early_stop_threshold, stop
                print('early stopping!')
                break
            potential_user_nodes = random.sample(set(G[question_node]), 2)
            user_node = potential_user_nodes[0] if not is_same_edge(x, y, potential_user_nodes[0], curr_pos) else potential_user_nodes[1]
            if random.random() < teleportation_alpha:
                curr_pos = starting_pos
            else:
                curr_pos = user_node
        except ValueError:
            # encouter valueError during random.sample when population is smaller than 2
            # This only happens if we reached a deadend
            # simply teleport back
            curr_pos = starting_pos
    # calculate distribution
    tot_visits = sum(reacheable_count.values())
    # sort visits by counts
    all_visits = sorted(reacheable_count.items(), key=lambda x: x[1])
    return [(i[0], i[1] / tot_visits) for i in all_visits]



In [26]:
all_edges = list(G.edges())
all_edges[:10]

[(61, 7045118),
 (61, 7069298),
 (61, 7093008),
 (61, 7096418),
 (61, 7187368),
 (61, 7571688),
 (61, 8626588),
 (61, 9565248),
 (61, 13287568),
 (61, 13598978)]

In [27]:
print(G[61])
print(list(G[61]))

{7045118: {'weight': 1.0}, 7069298: {'weight': 1.0}, 7093008: {'weight': 1.0}, 7096418: {'weight': 1.0}, 7187368: {'weight': 1.0}, 7571688: {'weight': 1.0}, 8626588: {'weight': 1.0}, 9565248: {'weight': 1.0}, 13287568: {'weight': 1.0}, 13598978: {'weight': 1.0}}
[7045118, 7069298, 7093008, 7096418, 7187368, 7571688, 8626588, 9565248, 13287568, 13598978]


In [29]:
test_distr = random_walk_on_edge(61, 7045118)
print(test_distr[:10])
print(len(test_distr))
print([i for i in test_distr if i[0] == 7045118])

[(9368798, 0.0010162601626016261), (7611138, 0.0010162601626016261), (15525668, 0.0010162601626016261), (7240868, 0.0010162601626016261), (7779168, 0.0010162601626016261), (21021708, 0.0010162601626016261), (7593448, 0.0010162601626016261), (8324948, 0.0010162601626016261), (8025148, 0.0010162601626016261), (15154978, 0.0010162601626016261)]
299
[]


In [31]:
import random
random.sample(set(G[61]), 2)

[7093008, 7096418]

In [32]:
1 in set([1, 2, 3])

True